# Going the (Manhattan) distance

* https://adventofcode.com/2022/day/15

It's all about Manhattan distances again, and the grid reminds me about [Day 6, 2018](../2018/Day%2006.ipynb), so I'm busting out the [`scipy.spatial.distance` module](https://docs.scipy.org/doc/scipy/reference/spatial.distance.html) again.

For part 1, we only need to know how many points on a specific line are covered by the sensors. Each sensor-beacon combo entails a specific Manhattan distance. Using `cdist()` you can then calculate the distance between all points on that line to all the sensors, and only take those points that are equal or lower to the sensor-beacon distance. Then count the number of points that have at least one sensor covering the point, and we are done.

In [1]:
import re
from typing import Final, Self

import numpy as np
from scipy.spatial import distance


NOT_NUMBERS_OR_NL: Final[re.Pattern[str]] = re.compile(r"[^\n\d-]+")


class SensorReport:
    def __init__(self, sensors: np.ndarray, beacons: np.ndarray):
        self.sensors = sensors
        self.beacons = beacons
        self.reach = np.sum(np.abs(self.sensors - self.beacons), axis=1)

    @classmethod
    def from_report_text(cls, report: str) -> Self:
        data = NOT_NUMBERS_OR_NL.sub(" ", report)
        coords = np.genfromtxt(data.splitlines(), delimiter=" ", dtype=int)
        return cls(coords[:, :2], coords[:, 2:])

    def count_line_cover(self, y: int) -> int:
        sensors, reach, beacons = self.sensors, self.reach, self.beacons
        # array with all possible intersection points on line y
        x = np.arange((sensors[:, 0] - reach).min(), (sensors[:, 0] + reach).max() + 1)
        line = np.pad(x[:, None], ((0, 0), (0, 1)), constant_values=y)
        # the distances from each sensor to each point on the line
        distances = distance.cdist(sensors, line, "cityblock")
        # what points are in reach of at least one sensor?
        reached = np.any(distances <= reach[:, None], axis=0)
        # How many beacons already lie on the line (and so can be discounted)
        beacon_count = np.unique(beacons[beacons[:, 1] == y], axis=0).shape[0]
        # how many points are covered, minus the number of beacons
        return np.sum(reached) - beacon_count


example = SensorReport.from_report_text(
    """\
Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3
"""
)
assert example.count_line_cover(10) == 26

In [2]:
import aocd


report = SensorReport.from_report_text(aocd.get_data(day=15, year=2022))
print("Part 1:", report.count_line_cover(2000000))

Part 1: 5525990


## Part 2, find all the points

We can't  just use the `cdist()` function here, because calculating the distance of 3 dozen sensors to 16 trillion (16 * 10 ^ 12) is a bit more than even scipy can handle efficiently.

Instead, you can take the y intercepts of each of the 4 diagonals of each sensor range **plus or minus 1** (depending on which side of the sensor you are); the beacon location will lie on an intersection of two such diagonals. The diagonals are always at a 45º angle, so have a slope of either 1 or -1, and we only need to look at those diagonals that lie between two scanner ranges (so the upper diagonal of one matches the lower diagonal of another).

Once you have determined the shared positive and negative slope intercepts, you can get the intersections points. There can be more than one such a point, but with only a handful *now* we can use the `cdist()` function to find the one point that's not within any of the ranges.

In [3]:
from itertools import product


def locate_distress_tuning_freq(report: SensorReport, max_xy: int) -> int:
    sensors, reach = report.sensors, report.reach
    # intercepts for the diagonal **just outside** each sensing range
    # in pos, pos, neg, neg form.
    yintercepts = np.stack(
        [
            sensors[:, 1] - sensors[:, 0] - reach - 1,
            sensors[:, 1] - sensors[:, 0] + reach + 1,
            sensors[:, 1] + sensors[:, 0] - reach - 1,
            sensors[:, 1] + sensors[:, 0] + reach + 1,
        ],
        axis=1,
    )

    # combinations of each set of slopes with each other set
    combos = np.stack(np.triu_indices(len(report.sensors), k=1), axis=-1)
    paired = yintercepts[combos]

    # paired sensors with matching intercepts, for positive and negative slopes
    # this matches (lower pos, lower neg) of one sensor with (upper pos, upper neg)
    # of the other sensor, and vice versa, into a [bool, bool] row per combination
    pos = np.unique(
        np.concatenate(
            [
                paired[paired[:, 0, 0] == paired[:, 1, 1]][:, 0, 0],
                paired[paired[:, 0, 1] == paired[:, 1, 0]][:, 0, 1],
            ]
        )
    )
    neg = np.unique(
        np.concatenate(
            [
                paired[paired[:, 0, 2] == paired[:, 1, 3]][:, 0, 2],
                paired[paired[:, 0, 3] == paired[:, 1, 2]][:, 0, 3],
            ]
        )
    )
    combined = np.transpose([np.tile(pos, len(neg)), np.repeat(neg, len(pos))])
    x = (-combined[:, 0] + combined[:, 1]) // 2
    y = x + combined[:, 0]
    points = np.stack([x, y], axis=1)
    unscanned = points[
        ~np.any(
            (distance.cdist(sensors, points, "cityblock") <= reach[:, None]), axis=0
        )
    ].reshape(-1)
    assert unscanned.size == 2
    return unscanned[0] * 4000000 + unscanned[1]


assert locate_distress_tuning_freq(example, 20) == 56000011

In [4]:
print("Part 2:", locate_distress_tuning_freq(report, 4000000))

Part 2: 11756174628223
